# The different algos for QR factorization

In [27]:
import numpy as np
import random
import itertools
import copy
import operator
import time
from math import copysign
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow as pa
import pyarrow.feather as feather

Create Test Matrix

In [28]:
mat = np.array([[3, 4, 2, 1],
                [5, 6, 7, 8],
                [3, 4, 5, 1],
                [1, 2, 3, 4]], dtype=float)

arrow_mat = pd.DataFrame(mat, columns = list(range(4)))
feather.write_feather(arrow_mat, 'feathers/starting_mat')

## Gram Schmidt

In [29]:
def gram_schmidt_process(A):
    """Perform QR decomposition of matrix A using Gram-Schmidt process."""
    (num_rows, num_cols) = np.shape(A)

    # Initialize empty orthogonal matrix Q.
    Q = np.empty([num_rows, num_rows])
    cnt = 0

    # Compute orthogonal matrix Q.
    for a in A.T:
        u = np.copy(a)
        for i in range(0, cnt):
            proj = np.dot(np.dot(Q[:, i].T, a), Q[:, i])
            u -= proj

        e = u / np.linalg.norm(u)
        Q[:, cnt] = e

        cnt += 1  # Increase columns counter.

    # Compute upper triangular matrix R.
    R = np.dot(Q.T, A)

    return (Q, R)


In [30]:
q = gram_schmidt_process(mat)[0]
r = gram_schmidt_process(mat)[0]

arrow_gs_q = pd.DataFrame(q, columns = list(range(4)))
feather.write_feather(arrow_gs_q, 'feathers/gs_q')

arrow_gs_r = pd.DataFrame(r, columns = list(range(4)))
feather.write_feather(arrow_gs_r, 'feathers/gs_r')

print("Q", q)
print("R", r)

Q [[ 4.52267017e-01  2.13200716e-01 -8.66025404e-01 -1.08779196e-16]
 [ 7.53778361e-01 -4.26401433e-01  2.88675135e-01  4.08248290e-01]
 [ 4.52267017e-01  2.13200716e-01  2.88675135e-01 -8.16496581e-01]
 [ 1.50755672e-01  8.52802865e-01  2.88675135e-01  4.08248290e-01]]
R [[ 4.52267017e-01  2.13200716e-01 -8.66025404e-01 -1.08779196e-16]
 [ 7.53778361e-01 -4.26401433e-01  2.88675135e-01  4.08248290e-01]
 [ 4.52267017e-01  2.13200716e-01  2.88675135e-01 -8.16496581e-01]
 [ 1.50755672e-01  8.52802865e-01  2.88675135e-01  4.08248290e-01]]


## Householder

In [31]:
def householder_reflection(A):
    """Perform QR decomposition of matrix A using Householder reflection."""
    (num_rows, num_cols) = np.shape(A)

    # Initialize orthogonal matrix Q and upper triangular matrix R.
    Q = np.identity(num_rows)
    R = np.copy(A)

    # Iterative over column sub-vector and
    # compute Householder matrix to zero-out lower triangular matrix entries.
    for cnt in range(num_rows - 1):
        x = R[cnt:, cnt]

        e = np.zeros_like(x)
        e[0] = copysign(np.linalg.norm(x), -A[cnt, cnt])
        u = x + e
        v = u / np.linalg.norm(u)

        Q_cnt = np.identity(num_rows)
        Q_cnt[cnt:, cnt:] -= 2.0 * np.outer(v, v)

        R = np.dot(Q_cnt, R)
        Q = np.dot(Q, Q_cnt.T)

    return (Q, R)


In [32]:
q = householder_reflection(mat)[0]
r = householder_reflection(mat)[1]

arrow_house_q = pd.DataFrame(q, columns = list(range(4)))
feather.write_feather(arrow_house_q, 'feathers/house_q')

arrow_house_r = pd.DataFrame(r, columns = list(range(4)))
feather.write_feather(arrow_house_r, 'feathers/house_r')

print("Q", q)
print("R", r)

Q [[ 4.52267017e-01  2.13200716e-01 -8.66025404e-01 -1.19899678e-16]
 [ 7.53778361e-01 -4.26401433e-01  2.88675135e-01  4.08248290e-01]
 [ 4.52267017e-01  2.13200716e-01  2.88675135e-01 -8.16496581e-01]
 [ 1.50755672e-01  8.52802865e-01  2.88675135e-01  4.08248290e-01]]
R [[ 6.63324958e+00  8.44231765e+00  8.89458467e+00  7.53778361e+00]
 [ 4.95342708e-16  8.52802865e-01  1.06600358e+00  4.26401433e-01]
 [-1.06663494e-15  4.85153042e-17  2.59807621e+00  2.88675135e+00]
 [-4.30584319e-16 -5.23972308e-17  6.19851865e-16  4.08248290e+00]]


## Givens

In [33]:

def givens_rotation(A):
    """Perform QR decomposition of matrix A using Givens rotation."""
    (num_rows, num_cols) = np.shape(A)

    # Initialize orthogonal matrix Q and upper triangular matrix R.
    Q = np.identity(num_rows)
    R = np.copy(A)

    # Iterate over lower triangular matrix.
    (rows, cols) = np.tril_indices(num_rows, -1, num_cols)
    for (row, col) in zip(rows, cols):

        # Compute Givens rotation matrix and
        # zero-out lower triangular matrix entries.
        if R[row, col] != 0:
            (c, s) = _givens_rotation_matrix_entries(R[col, col], R[row, col])

            G = np.identity(num_rows)
            G[[col, row], [col, row]] = c
            G[row, col] = s
            G[col, row] = -s

            R = np.dot(G, R)
            Q = np.dot(Q, G.T)

    return (Q, R)


def _givens_rotation_matrix_entries(a, b):
    """Compute matrix entries for Givens rotation."""
    r = np.linalg.norm([a, b])
    c = a/r
    s = -b/r

    return (c, s)


In [35]:
q = givens_rotation(mat)[0]
r = givens_rotation(mat)[1]

arrow_givens_q = pd.DataFrame(q, columns = list(range(4)))
feather.write_feather(arrow_givens_q, 'feathers/givens_q')

arrow_givens_r = pd.DataFrame(r, columns = list(range(4)))
feather.write_feather(arrow_givens_r, 'feathers/givens_r')

print("Q", q)
print("R", r)

Q [[ 4.52267017e-01  2.13200716e-01 -8.66025404e-01 -2.00642589e-16]
 [ 7.53778361e-01 -4.26401433e-01  2.88675135e-01 -4.08248290e-01]
 [ 4.52267017e-01  2.13200716e-01  2.88675135e-01  8.16496581e-01]
 [ 1.50755672e-01  8.52802865e-01  2.88675135e-01 -4.08248290e-01]]
R [[ 6.63324958e+00  8.44231765e+00  8.89458467e+00  7.53778361e+00]
 [ 3.40477089e-17  8.52802865e-01  1.06600358e+00  4.26401433e-01]
 [ 3.85570459e-17 -1.57042915e-17  2.59807621e+00  2.88675135e+00]
 [ 9.83872928e-17 -8.89788665e-18  6.26105051e-17 -4.08248290e+00]]


## Test Efficiencies

In [36]:
def compare(V, step, reps):
    df = pd.DataFrame(columns=['i', 'time', 'algorithm'])

    for i in range(2, V, step):
            
        sum_givens = 0
        sum_gs = 0
        sum_house = 0

        for rep in range(reps):

            mat = np.random.randint(10, size=(i, i)).astype(float)
            
            start_givens = time.time()
            givens_rotation(mat)
            end_givens = time.time()
            sum_givens += end_givens - start_givens


            start_gs = time.time()
            gram_schmidt_process(mat)
            end_gs = time.time()
            sum_gs += end_gs - start_gs

            
            start_house = time.time()
            householder_reflection(mat)
            end_house = time.time()
            sum_house += end_house - start_house


        avg_givens = sum_givens / reps
        avg_gs = sum_gs / reps
        avg_house = sum_house / reps

        df = df.append({'i': i, 'time': avg_givens, 'algorithm': "Givens"}, ignore_index=True)
        df = df.append({'i': i, 'time': avg_gs, 'algorithm': "Gram-Schmidt"}, ignore_index=True)
        df = df.append({'i': i, 'time': avg_house, 'algorithm': "Householder"}, ignore_index=True)

        print("finished", i)
        


    sns.lineplot(x=df.i, y=df.time, hue=df.algorithm)
    plt.xlabel("Size of matrix")
    plt.ylabel("Avg Time of Trial (sec)")
    plt.legend()
    plt.savefig('graphs/times.png')

compare(100, 5, 30)

/tmp/ipykernel_14758/2410068899.py:17: RuntimeWarning: invalid value encountered in true_divide
  v = u / np.linalg.norm(u)
/tmp/ipykernel_14758/464808251.py:16: RuntimeWarning: invalid value encountered in true_divide
  e = u / np.linalg.norm(u)


finished 2
finished 7
finished 12
finished 17
finished 22
finished 27
finished 32


KeyboardInterrupt: 